In [1]:
# импортируем необходимые библиотеки
import numpy as np
import pandas as pd

# записываем CSV-файл в объект DataFrame
data = pd.read_csv("C:/Trees/RF_classification.csv", encoding='cp1251', sep=';')

# смотрим данные
data

,age,gender,default
0,70,Мужской,Да
1,64,Мужской,Нет
2,69,Женский,Да
3,68,Мужской,Нет
4,65,Женский,Нет


In [2]:
# выполняем дамми-кодирование
print("Исходные переменные:\n", list(data.columns), "\n")
data_dummies = pd.get_dummies(data)
print("Переменные после get_dummies:\n", list(data_dummies.columns))

Исходные переменные:
 ['age', 'gender', 'default'] 

Переменные после get_dummies:
 ['age', 'gender_Женский', 'gender_Мужской', 'default_Да', 'default_Нет']


In [3]:
# создаем массив меток зависимой переменной
y = data_dummies.loc[:, 'default_Да']
# удаляем из массива признаков дамми-переменные
# default_Нет и default_Да, поскольку они
# представляют собой зависимую переменную
data_dummies.drop('default_Нет', axis=1, inplace=True)
data_dummies.drop('default_Да', axis=1, inplace=True)
# создаем массив признаков
X = data_dummies

In [4]:
# импортируем класс RandomForestClassifier
from sklearn.ensemble import RandomForestClassifier
# создаем экземпляр класса RandomForestClassifier
forest=RandomForestClassifier(n_estimators=10, random_state=152, 
                              oob_score=True, n_jobs=-1)
# подгоняем модель
forest.fit(X, y)
# вычисляем правильность по методу OOB
print("Правильность по методу OOB: {:.2f}".format(forest.oob_score_))

Правильность по методу OOB: 0.40


In [5]:
# прогнозируем классы зависимой переменной по методу OOB
oob_predictions=np.argmax(forest.oob_decision_function_, axis=1)
# преобразуем прогнозы в объект DataFrame
oob_predictions=pd.DataFrame(oob_predictions, 
                             index=data.index, columns=['OOB_Predictions'])
# конкатенируем массив признаков, массив меток и OOB прогнозы
oob_results=pd.concat([X, y, oob_predictions], axis=1)
oob_results

,age,gender_Женский,gender_Мужской,default_Да,OOB_Predictions
0,70,0,1,1,0
1,64,0,1,0,0
2,69,1,0,1,0
3,68,0,1,0,1
4,65,1,0,0,0


In [6]:
# прогнозируем вероятности классов по методу OOB
print("Вероятности, спрогнозированные по методу OOB:\n{}".
      format(forest.oob_decision_function_))

Вероятности, спрогнозированные по методу OOB:
[[ 0.8         0.2       ]
 [ 1.          0.        ]
 [ 0.8         0.2       ]
 [ 0.          1.        ]
 [ 0.66666667  0.33333333]]


In [7]:
# вычисляем правильность по обычному методу
print("Правильность по обычному методу: {:.3f}".format(forest.score(X, y)))

Правильность по обычному методу: 1.000


In [8]:
# прогнозируем классы зависимой переменной по обычному методу
predictions=forest.predict(X)
# преобразуем прогнозы в объект DataFrame
predictions=pd.DataFrame(predictions, 
                         index=data.index, columns=['Predictions'])
# конкатенируем массив признаков, массив меток и прогнозы
results=pd.concat([X, y, predictions], axis=1)
results

,age,gender_Женский,gender_Мужской,default_Да,Predictions
0,70,0,1,1,1
1,64,0,1,0,0
2,69,1,0,1,1
3,68,0,1,0,0
4,65,1,0,0,0


In [9]:
# прогнозируем вероятности классов по обычному методу
print("Вероятности, спрогнозированные по обычному методу:\n{}".
      format(forest.predict_proba(X)))

Вероятности, спрогнозированные по обычному методу:
[[ 0.4  0.6]
 [ 1.   0. ]
 [ 0.4  0.6]
 [ 0.7  0.3]
 [ 0.9  0.1]]


In [10]:
# вычисляем листовые вероятности каждого дерева
# для первого наблюдения 
for tree_in_forest in forest.estimators_:    
    print(tree_in_forest.predict_proba(X)[0])

[ 1.  0.]
[ 0.  1.]
[ 1.  0.]
[ 0.  1.]
[ 0.  1.]
[ 0.  1.]
[ 1.  0.]
[ 0.  1.]
[ 0.  1.]
[ 1.  0.]


In [11]:
# записываем CSV-файл в объект DataFrame
data = pd.read_csv("C:/Trees/RF_regression.csv", encoding='cp1251', sep=';')

# смотрим данные
data

,age,gender,days_of_delinquency
0,70,Мужской,18
1,64,Мужской,15
2,69,Женский,17
3,68,Мужской,19
4,65,Женский,15
5,63,Мужской,16
6,55,Женский,13
7,60,Мужской,14
8,67,Мужской,16
9,68,Женский,14


In [12]:
# выполняем дамми-кодирование
print("Исходные переменные:\n", list(data.columns), "\n")
data_dummies = pd.get_dummies(data)
print("Переменные после get_dummies:\n", list(data_dummies.columns))

Исходные переменные:
 ['age', 'gender', 'days_of_delinquency'] 

Переменные после get_dummies:
 ['age', 'days_of_delinquency', 'gender_Женский', 'gender_Мужской']


In [13]:
# создаем массив значений зависимой переменной
y = data_dummies.loc[:, 'days_of_delinquency']
# удаляем из массива признаков переменную
# days_of_delinquency, поскольку она
# представляет собой зависимую переменную
data_dummies.drop('days_of_delinquency', axis=1, inplace=True)
# создаем массив признаков
X = data_dummies

In [14]:
# импортируем класс RandomForestRegressor
from sklearn.ensemble import RandomForestRegressor
# создаем экземпляр класса RandomForestRegressor
forest=RandomForestRegressor(n_estimators=10, random_state=152, 
                              oob_score=True, n_jobs=-1)
# подгоняем модель
forest.fit(X, y)
# вычисляем R-квадрат по методу OOB
print("R-квадрат по методу OOB: {:.2f}".format(forest.oob_score_))

R-квадрат по методу OOB: -0.36


In [15]:
# прогнозируем значения зависимой переменной по методу OOB
oob_predictions=forest.oob_prediction_
# преобразуем прогнозы в объект DataFrame
oob_predictions=pd.DataFrame(oob_predictions, 
                             index=data.index, columns=['OOB_Predictions'])
# конкатенируем массив признаков, массив значений 
# зависимой переменной и OOB прогнозы
oob_results=pd.concat([X, y, oob_predictions], axis=1)
oob_results

,age,gender_Женский,gender_Мужской,days_of_delinquency,OOB_Predictions
0,70,0,1,18,17.800000
1,64,0,1,15,16.000000
2,69,1,0,17,16.000000
3,68,0,1,19,14.666667
4,65,1,0,15,14.600000
5,63,0,1,16,15.000000
6,55,1,0,13,14.333333
7,60,0,1,14,15.800000
8,67,0,1,16,17.750000
9,68,1,0,14,17.666667


In [16]:
# вычисляем сумму квадратов отклонений фактических значений
# зависимой переменной от ее среднего значения
TSS=((oob_results['days_of_delinquency']-oob_results['days_of_delinquency'].mean())**2).sum()
# вычисляем сумму квадратов отклонений фактических значений 
# зависимой переменной от спрогнозированных, при этом каждое 
# спрогнозированное значение – результат усреднения средних
# значений, вычисленных деревьями по OOB выборкам 
RSS=((oob_results['days_of_delinquency']-oob_results['OOB_Predictions'])**2).sum()
# вычисляем R-квадрат по методу OOB
print("R-квадрат по методу OOB, вычисленный вручную: {:.2f}".format(1-(RSS/TSS)))

R-квадрат по методу OOB, вычисленный вручную: -0.36


In [17]:
# вычисляем R-квадрат по обычному методу
print("R-квадрат по обычному методу: {:.2f}".format(forest.score(X, y)))

R-квадрат по обычному методу: 0.85


In [18]:
# прогнозируем значения зависимой переменной по обычному методу
predictions=forest.predict(X)
# преобразуем прогнозы в объект DataFrame
predictions=pd.DataFrame(predictions, 
                         index=data.index, columns=['Predictions'])
# конкатенируем массив признаков, массив значений 
# зависимой переменной и прогнозы
results=pd.concat([X, y, predictions], axis=1)
results

,age,gender_Женский,gender_Мужской,days_of_delinquency,Predictions
0,70,0,1,18,17.9
1,64,0,1,15,15.3
2,69,1,0,17,16.5
3,68,0,1,19,17.7
4,65,1,0,15,14.8
5,63,0,1,16,15.9
6,55,1,0,13,13.4
7,60,0,1,14,14.9
8,67,0,1,16,16.7
9,68,1,0,14,15.1


In [19]:
# вычисляем сумму квадратов отклонений фактических значений
# зависимой переменной от ее среднего значения
TSS=((results['days_of_delinquency']-results['days_of_delinquency'].mean())**2).sum()
# вычисляем сумму квадратов отклонений фактических значений 
# зависимой переменной от спрогнозированных, при этом каждое 
# спрогнозированное значение – результат усреднения средних
# значений, вычисленных деревьями по всем бутстреп-выборкам 
RSS=((results['days_of_delinquency']-results['Predictions'])**2).sum()
# вычисляем R-квадрат по обычному методу
print("R-квадрат по обычному методу, вычисленный вручную: {:.2f}".format(1-(RSS/TSS)))

R-квадрат по обычному методу, вычисленный вручную: 0.85


In [20]:
from sklearn.tree import _tree

# пишем функцию, которая вычисляет глубину терминальных 
# узлов в отдельном дереве
def leaf_depths(tree, node_id = 0):

    '''
    tree.children_left и tree.children_right записывают идентификационные
    номера левого и правого узлов-потомков для данного узла
    '''

    left_child = tree.children_left[node_id]
    right_child = tree.children_right[node_id]

    '''
    если данный узел является терминальным, 
    то оба дочерних узла будут иметь значение _tree.TREE_LEAF,
    что позволяет нам проверить, является ли данный узел терминальным
    '''

    if left_child == _tree.TREE_LEAF:
      
       '''
       задать глубину терминальных узлов равной 0
       '''
      
       depths = np.array([0])

    else:
     
        '''
        получить значения глубины узлов-потомков
        и увеличить их на 1
        '''
        
        left_depths = leaf_depths(tree, left_child) + 1
        right_depths = leaf_depths(tree, right_child) + 1
 
        depths = np.append(left_depths, right_depths)
 
    return depths


In [21]:
# пишем функцию, которая вычисляет минимальное количество наблюдений 
# в терминальных узлах отдельного дерева
def leaf_samples(tree, node_id = 0):
    
    left_child = tree.children_left[node_id]
    right_child = tree.children_right[node_id]

    if left_child == _tree.TREE_LEAF:
        
         samples = np.array([tree.n_node_samples[node_id]])

    else:
        
        left_samples = leaf_samples(tree, left_child)
        right_samples = leaf_samples(tree, right_child)

        samples = np.append(left_samples, right_samples)

    return samples


In [22]:
# пишем функцию, которая строит гистограмму распределения 
# глубин терминальных узлов и гистограмму распределения 
# количества наблюдений в терминальных узлах по ансамблю
def draw_ensemble(ensemble):

    plt.figure(figsize=(8,8))
    plt.subplot(211)

    depths_all = np.array([], dtype=int)

    for x in ensemble.estimators_:
        tree = x.tree_
        depths = leaf_depths(tree)
        depths_all = np.append(depths_all, depths)
        plt.hist(depths, histtype='step', color='#ddaaff', 
                 bins=range(min(depths), max(depths)+1))

    plt.hist(depths_all, histtype='step', color='#9933ff', 
             bins=range(min(depths_all), max(depths_all)+1), 
             weights=np.ones(len(depths_all))/len(ensemble.estimators_), 
             linewidth=2)
    plt.xlabel("Глубина терминальных узлов")
   
    samples_all = np.array([], dtype=int)
    
    plt.subplot(212)
    
    for x in ensemble.estimators_:
        tree = x.tree_
        samples = leaf_samples(tree)
        samples_all = np.append(samples_all, samples)
        plt.hist(samples, histtype='step', color='#aaddff', 
                 bins=range(min(samples), max(samples)+1))
    
    plt.hist(samples_all, histtype='step', color='#3399ff', 
             bins=range(min(samples_all), max(samples_all)+1), 
             weights=np.ones(len(samples_all))/len(ensemble.estimators_), 
             linewidth=2)
    plt.xlabel("Количество наблюдений в терминальных узлах")
    
    plt.show()
